In [ ]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import ngsolve  # noqa: F401
else:
    try:
        import ngsolve
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/ngsolve-install.sh" -O "/tmp/ngsolve-install.sh" && bash "/tmp/ngsolve-install.sh"
        import ngsolve  # noqa: F401

In [ ]:
# Adapted from https://ngsolve.org/docu/latest/i-tutorials/unit-1.7-helmholtz/pml.html

In [ ]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.webgui import Draw

In [ ]:
geo = SplineGeometry()
geo.AddCircle((0, 0), 1.4, leftdomain=2, bc="outerbnd")
geo.AddCircle((0, 0), 1, leftdomain=1, rightdomain=2, bc="innerbnd")
geo.SetMaterial(1, "inner")
geo.SetMaterial(2, "pmlregion")
mesh = Mesh(geo.GenerateMesh(maxh=0.1))
mesh.Curve(3)
mesh.SetPML(pml.Radial(rad=1, alpha=1j, origin=(0, 0)), "pmlregion")

In [ ]:
Draw(mesh)

In [ ]:
fes = H1(mesh, order=4, complex=True)

In [ ]:
u = fes.TrialFunction()
v = fes.TestFunction()

In [ ]:
omega = 10
a = BilinearForm(fes)
a += grad(u) * grad(v) * dx - omega**2 * u * v * dx
a += -1j * omega * u * v * ds("outerbnd")
a.Assemble()

In [ ]:
f = exp(-20**2 * ((x - 0.3) * (x - 0.3) + y * y))
b = LinearForm(fes)
b += f * v * dx
b.Assemble()

In [ ]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse() * b.vec

In [ ]:
Draw(gfu)

In [ ]:
geo = SplineGeometry()
pnums = [
    geo.AddPoint(x, y, maxh=maxh) for x, y, maxh in [
        (-2, 0, 1), (-1, 0, 1), (-0.2, 0, 0.01), (-0.2, -0.8, 1), (0.2, -0.8, 1), (0.2, 0, 0.01),
        (1, 0, 1), (2, 0, 1), (2, 2, 1), (0, 2, 1), (-2, 2, 1), (1, 1, 1), (0, 1, 1), (-1, 1, 1)]]
lines = [
    (0, 1, "dir", 2, 0), (1, 2, "dir", 1, 0), (2, 3, "dir", 1, 0), (3, 4, "dir", 1, 0), (4, 5, "dir", 1, 0),
    (5, 6, "dir", 1, 0), (6, 7, "dir", 2, 0)]
curves = [
    (7, 8, 9, "outer", 2, 0), (9, 10, 0, "outer", 2, 0), (6, 11, 12, "inner", 1, 2), (12, 13, 1, "inner", 1, 2)]
for p1, p2, bc, left, right in lines:
    geo.Append(["line", pnums[p1], pnums[p2]], bc=bc, leftdomain=left, rightdomain=right)
for p1, p2, p3, bc, left, right in curves:
    geo.Append(["spline3", pnums[p1], pnums[p2], pnums[p3]], bc=bc, leftdomain=left, rightdomain=right)
geo.SetMaterial(1, "air")
geo.SetMaterial(2, "pml")

In [ ]:
mesh = Mesh(geo.GenerateMesh(maxh=0.1))
mesh.Curve(5)
mesh.SetPML(pml.Radial(rad=1, alpha=1j, origin=(0, 0)), "pml")

In [ ]:
Draw(mesh)

In [ ]:
fes = H1(mesh, order=4, complex=True, dirichlet="dir")

In [ ]:
u = fes.TrialFunction()
v = fes.TestFunction()

In [ ]:
a = BilinearForm(fes, symmetric=True)
a += grad(u) * grad(v) * dx
a.Assemble()

In [ ]:
m = BilinearForm(fes, symmetric=True)
m += u * v * dx
m.Assemble()

In [ ]:
u = GridFunction(fes, multidim=50, name="resonances")

In [ ]:
lam = ArnoldiSolver(a.mat, m.mat, fes.FreeDofs(), u.vecs, shift=400)
lamr = [sqrt(lam_).real for lam_ in lam]
lami = [sqrt(lam_).imag for lam_ in lam]
print(lamr)
print(lami)

In [ ]:
# Adapted from https://github.com/JSchoeberl/iFEM/blob/master/primal/first_example.ipynb

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo

In [ ]:
from netgen.occ import unit_square
DrawGeo(unit_square.shape)

In [ ]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))
Draw(mesh);

In [ ]:
fes = H1(mesh, order=3, dirichlet=".*")
a = BilinearForm(fes)
f = LinearForm(fes)

In [ ]:
u = fes.TrialFunction()
v = fes.TestFunction()

In [ ]:
a += grad(u) * grad(v) * dx
a.Assemble()

In [ ]:
funcf = 50 * x * y
f += funcf * v * dx
f.Assemble();

In [ ]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec
Draw(gfu);

In [ ]:
# Example of 3D visualization with widget

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import unit_cube

In [ ]:
mesh = Mesh(unit_cube.GenerateMesh(maxh=1))
Draw(mesh)